In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import statsmodels.api as sm

from sklearn.linear_model import LinearRegression

In [2]:
data= pd.read_csv('Multiple linear regression.csv')
data.head()

,SAT,GPA,"Rand 1,2,3"
0,1714,2.40,1
1,1664,2.52,3
2,1760,2.54,3
3,1685,2.74,3
4,1693,2.83,2


In [3]:
data.describe()

,SAT,GPA,"Rand 1,2,3"
count,84.000000,84.000000,84.000000
mean,1845.273810,3.330238,2.059524
std,104.530661,0.271617,0.855192
min,1634.000000,2.400000,1.000000
25%,1772.000000,3.190000,1.000000
50%,1846.000000,3.380000,2.000000
75%,1934.000000,3.502500,3.000000
max,2050.000000,3.810000,3.000000


## Multiple Linear Regression Using Sklearn

In [4]:
x, y= data[['SAT', 'Rand 1,2,3']], data['GPA']

In [5]:
multiple_reg= LinearRegression().fit(x,y)

In [6]:
Intercept= multiple_reg.intercept_
Coefficients= multiple_reg.coef_
R_squared= multiple_reg.score(x,y)
print(" Intercept= ",Intercept, "\n", "Coefficients=", 'SAT: ', Coefficients[0],' Rand 1,2,3: ', Coefficients[1], "\n","R_squared=", R_squared)

 Intercept=  0.29603261264909486 
 Coefficients= SAT:  0.0016535418013456692  Rand 1,2,3:  -0.00826982252316037 
 R_squared= 0.40668119528142843


### Formula for Adjusted R^2

In [7]:
n= x.shape[0]
p= x.shape[1]


Adj_R_squared= 1-(1-R_squared)*(n-1)/(n-p-1)
Adj_R_squared

0.39203134825134023

### Feature selection

as adj_r_2 is less than r2, so this means there is 1 or more featsure with few or no predictive power, so we use feature selection to determine the effective features.

In [8]:
from sklearn.feature_selection import f_regression

In [9]:
f_regression(x,y)

(array([56.04804786,  0.17558437]), array([7.19951844e-11, 6.76291372e-01]))

In [10]:
p_values= f_regression(x,y)[1].round(3)
p_values

array([0.   , 0.676])

### Standardization (feature scaling)

In [11]:
from sklearn.preprocessing import StandardScaler

In [12]:
scaler= StandardScaler()
scaler.fit(x)

StandardScaler()

In [13]:
scaler.get_params()

{'copy': True, 'with_mean': True, 'with_std': True}

In [14]:
x_scaled= scaler.transform(x)

In [15]:
x_scaled

array([[-1.26338288, -1.24637147],
       [-1.74458431,  1.10632974],
       [-0.82067757,  1.10632974],
       [-1.54247971,  1.10632974],
       [-1.46548748, -0.07002087],
       [-1.68684014, -1.24637147],
       [-0.78218146, -0.07002087],
       [-0.78218146, -1.24637147],
       [-0.51270866, -0.07002087],
       [ 0.04548499,  1.10632974],
       [-1.06127829,  1.10632974],
       [-0.67631715, -0.07002087],
       [-1.06127829, -1.24637147],
       [-1.28263094,  1.10632974],
       [-0.6955652 , -0.07002087],
       [ 0.25721362, -0.07002087],
       [-0.86879772,  1.10632974],
       [-1.64834403, -0.07002087],
       [-0.03150724,  1.10632974],
       [-0.57045283,  1.10632974],
       [-0.81105355,  1.10632974],
       [-1.18639066,  1.10632974],
       [-1.75420834,  1.10632974],
       [-1.52323165, -1.24637147],
       [ 1.23886453, -1.24637147],
       [-0.18549169, -1.24637147],
       [-0.5608288 , -1.24637147],
       [-0.23361183,  1.10632974],
       [ 1.68156984,

# Regression with scaled features

In [16]:
reg= LinearRegression()
reg.fit(x_scaled,y)

LinearRegression()

In [17]:
reg.coef_

array([ 0.17181389, -0.00703007])

### Summary table

In [18]:
reg_summary= pd.DataFrame([['Bias'], ['SAT'], ['Rand 1,2,3']], columns= ['Features'])

In [19]:
reg_summary['Weights']= reg.intercept_, reg.coef_[0], reg.coef_[1]

In [20]:
reg_summary

,Features,Weights
0,Bias,3.330238
1,SAT,0.171814
2,"Rand 1,2,3",-0.007030


### Making Prediction with wights

In [21]:
new_data= pd.DataFrame(data= [[1700,2], [1800,1]], columns= ['SAT', 'Rand 1,2,3'] )

In [22]:
new_data

,SAT,"Rand 1,2,3"
0,1700,2
1,1800,1


In [23]:
new_data_scaled= scaler.transform(new_data)
new_data_scaled

array([[-1.39811928, -0.07002087],
       [-0.43571643, -1.24637147]])

In [24]:
reg.predict(new_data_scaled)

array([3.09051403, 3.26413803])

In [25]:
new_data['Predicted']= reg.predict(new_data_scaled)

In [26]:
new_data

,SAT,"Rand 1,2,3",Predicted
0,1700,2,3.090514
1,1800,1,3.264138


### What if we remove 'Rans 1,2,3' variable

In [27]:
reg_simple= LinearRegression()
x_simple_matrix= x_scaled[:,0].reshape(-1,1)
reg_simple.fit(x_simple_matrix, y)

LinearRegression()

In [28]:
reg_simple.predict(new_data_scaled[:,0].reshape(-1,1))

array([3.08970998, 3.25527879])

## Multiple Linear Regression using statsmodels

In [29]:
x1= data[['SAT', 'Rand 1,2,3']]
y= data['GPA']
x= sm.add_constant(x1)

In [30]:
reg= sm.OLS(y,x)
multiple_reg= reg.fit()
multiple_reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    GPA   R-squared:                       0.407
Model:                            OLS   Adj. R-squared:                  0.392
Method:                 Least Squares   F-statistic:                     27.76
Date:                Sun, 10 Apr 2022   Prob (F-statistic):           6.58e-10
Time:                        09:51:04   Log-Likelihood:                 12.720
No. Observations:                  84   AIC:                            -19.44
Df Residuals:                      81   BIC:                            -12.15
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.2960      0.417      0.710      0.480      -0.533       1.125
SAT            0.0017      0.000      7.432      0.000       0.001       0.002
Rand 1,2,3    -0.0083      0.027     -0.304      0.762      -0.062       0.046
==============================================================================
Omnibus:                       12.992   Durbin-Watson:                   0.948
Prob(Omnibus):                  0.002   Jarque-Bera (JB):               16.364
Skew:                          -0.731   Prob(JB):                     0.000280
Kurtosis:                       4.594   Cond. No.                     3.33e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.33e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""